# General instructions

The goal of the project is to materialize a set of **exploratory workloads** over a real-world, large-scale,  open-domain KG: [WikiData](https://www.wikidata.org/wiki/Wikidata:Main_Page)

An exploratory workload  is composed by a set of queries, where each query is related to the information obtained previously.

An exploratory workload starts with a usually vague, open ended question, and does not assume the person issuing the workload has a clear understanding of the data contained in the target database or its structure.

Remeber that:

1. All the queries must run in the python notebook
2. You can use classes and properties only if you find them via a SPARQL query that must be present in the notebook
3. You do not delete useless queries. Keep everything that is synthatically valid 

```
?p <http://schema.org/name> ?name .
```
    
is the BGP returning a human-readable name of a property or a class in Wikidata.
    
    

In [1]:
## SETUP used later

from SPARQLWrapper import SPARQLWrapper, JSON


prefixString = """
##-64886f2f8a-##
PREFIX wd: <http://www.wikidata.org/entity/> 
PREFIX wdt: <http://www.wikidata.org/prop/direct/> 
PREFIX sc: <http://schema.org/>
"""

# select and construct queries
def run_query(queryString):
    to_run = prefixString + "\n" + queryString

    sparql = SPARQLWrapper("http://a256-gc1-02.srv.aau.dk:5820/sparql")
    sparql.setTimeout(300)
    sparql.setReturnFormat(JSON)
    sparql.setQuery(to_run)

    try :
       results = sparql.query()
       json_results = results.convert()
       if len(json_results['results']['bindings'])==0:
          print("Empty")
          return 0
    
       for bindings in json_results['results']['bindings']:
          print( [ (var, value['value'])  for var, value in bindings.items() ] )

       return len(json_results['results']['bindings'])

    except Exception as e :
        print("The operation failed", e)
    
# ASk queries
def run_ask_query(queryString):
    to_run = prefixString + "\n" + queryString

    sparql = SPARQLWrapper("http://a256-gc1-02.srv.aau.dk:5820/sparql")
    sparql.setTimeout(300)
    sparql.setReturnFormat(JSON)
    sparql.setQuery(to_run)

    try :
        return sparql.query().convert()

    except Exception as e :
        print("The operation failed", e)


# Companies Workflow Series ("Trademarks across the world") 

Consider the following exploratory information need:

> You are investigating different kinds of trademarks

## Useful URIs for the current workflow


The following are given:

| IRI           | Description   | Role      |
| -----------   | -----------   |-----------|
| `wdt:P1647`   | subproperty   | predicate |
| `wdt:P31`     | instance of   | predicate |
| `wdt:P279`    | subclass      | predicate |
| `wdt:P17`      | country       | predicate |
| `wdt:P27`      | citizenship   | predicate |
| `wd:Q167270`  | trademark     | node      |
| `wd:Q14091`   | iMac          | node      |
| `wd:Q312`     | Apple Inc.    | node      |



Also consider

```
?p wdt:P31/wdt:P279* wd:Q167270  . 
```

is the BGP to retrieve all **entities that are trademarks**

## Workload Goals

1. Identify the BGP for the company or person related to a given trademark

2. Identify the BGP to retrieve types or categories for a given trademark

3. What companies have the largest number of trademarks? 

4. What are the types of trademarks, how many trademarks exist for each type?

5. Analyze the number of trademarks across types, companies, and countries
 
   5.1 How many U.S. companies hold trademarks? In which sectors?
   
   5.2 How many people hold or are connected to trademarks? In which role?
   
   5.3 In which sector there is the highest number of trademarks?   


In [2]:
# start your workflow here

In [3]:
#the BGP to retrieve all entities that are trademarks
queryString = """
SELECT COUNT(?p)
WHERE { 


?p wdt:P31/wdt:P279* wd:Q167270  . 

} 
"""

print("Results")
run_query(queryString)

Results
[('callret-0', '44700')]


1

## Identify the BGP for the company or person related to a given trademark

In [4]:
## wd:Q14091	iMac	node

queryString = """
SELECT ?s ?sname ?p ?pname
WHERE { 

    ?s ?p wd:Q14091 . 
    # label
    ?p <http://schema.org/name> ?pname .
    ?s <http://schema.org/name> ?sname .
} 
LIMIT 100
"""

print("Results")
run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q306381'), ('sname', 'iMac G3'), ('p', 'http://www.wikidata.org/prop/direct/P279'), ('pname', 'subclass of')]
[('s', 'http://www.wikidata.org/entity/Q19837'), ('sname', 'Steve Jobs'), ('p', 'http://www.wikidata.org/prop/direct/P800'), ('pname', 'notable work')]
[('s', 'http://www.wikidata.org/entity/Q312'), ('sname', 'Apple Inc.'), ('p', 'http://www.wikidata.org/prop/direct/P1056'), ('pname', 'product or material produced')]
[('s', 'http://www.wikidata.org/entity/Q1050381'), ('sname', 'iMac G5'), ('p', 'http://www.wikidata.org/prop/direct/P279'), ('pname', 'subclass of')]
[('s', 'http://www.wikidata.org/entity/Q2407708'), ('sname', 'iMac G4'), ('p', 'http://www.wikidata.org/prop/direct/P279'), ('pname', 'subclass of')]
[('s', 'http://www.wikidata.org/entity/Q101439988'), ('sname', 'Apple M1'), ('p', 'http://www.wikidata.org/prop/direct/P1535'), ('pname', 'used by')]
[('s', 'http://www.wikidata.org/entity/Q406655'), ('sname', 'AirDrop'), ('

9

In [5]:
## wd:Q14091	iMac	node

queryString = """
SELECT ?s ?sname ?p ?pname
WHERE { 

    wd:Q14091 ?s ?p . 
    # label
    ?p <http://schema.org/name> ?pname .
    ?s <http://schema.org/name> ?sname .
} 
LIMIT 100
"""

print("Results")
run_query(queryString)

Results
[('s', 'http://www.wikidata.org/prop/direct/P176'), ('sname', 'manufacturer'), ('p', 'http://www.wikidata.org/entity/Q312'), ('pname', 'Apple Inc.')]
[('s', 'http://www.wikidata.org/prop/direct/P178'), ('sname', 'developer'), ('p', 'http://www.wikidata.org/entity/Q312'), ('pname', 'Apple Inc.')]
[('s', 'http://www.wikidata.org/prop/direct/P1889'), ('sname', 'different from'), ('p', 'http://www.wikidata.org/entity/Q306273'), ('pname', 'eMac')]
[('s', 'http://www.wikidata.org/prop/direct/P279'), ('sname', 'subclass of'), ('p', 'http://www.wikidata.org/entity/Q75687'), ('pname', 'Apple Macintosh')]
[('s', 'http://www.wikidata.org/prop/direct/P279'), ('sname', 'subclass of'), ('p', 'http://www.wikidata.org/entity/Q1258213'), ('pname', 'all-in-one PC')]
[('s', 'http://www.wikidata.org/prop/direct/P287'), ('sname', 'designed by'), ('p', 'http://www.wikidata.org/entity/Q164750'), ('pname', 'Jony ive')]
[('s', 'http://www.wikidata.org/prop/direct/P306'), ('sname', 'operating system'), 

9

In [6]:
# - Products developed by Apple Inc. - #
## wd:Q14091	iMac	node
## wdt:P178 -> developer 
## wd:Q312 -> Apple Inc.


queryString = """
SELECT COUNT(?product)
WHERE { 

    ?product wdt:P178 wd:Q312 .
    #?product wdt:P287 ?designers .
    
    # label
    ?product <http://schema.org/name> ?productname .
    #?designers <http://schema.org/name> ?designersname
} 

"""

print("Results")
run_query(queryString)

Results
[('callret-0', '668')]


1

In [7]:
## designers who worked for apple:
#designer
queryString = """
SELECT ?s ?sname ?p ?pname
WHERE { 
    ?s ?p wd:Q164750 .
    # label
    ?p <http://schema.org/name> ?pname .
    ?s <http://schema.org/name> ?sname .
} 

"""

print("Results")
run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q14091'), ('sname', 'iMac'), ('p', 'http://www.wikidata.org/prop/direct/P287'), ('pname', 'designed by')]
[('s', 'http://www.wikidata.org/entity/Q9479'), ('sname', 'iPod'), ('p', 'http://www.wikidata.org/prop/direct/P170'), ('pname', 'creator')]
[('s', 'http://www.wikidata.org/entity/Q817363'), ('sname', 'Benjamin Franklin Medal'), ('p', 'http://www.wikidata.org/prop/direct/P1346'), ('pname', 'winner')]
[('s', 'http://www.wikidata.org/entity/Q30587923'), ('sname', 'Designed by Apple in California'), ('p', 'http://www.wikidata.org/prop/direct/P50'), ('pname', 'author')]


4

In [8]:
## designers who worked for apple:
queryString = """
SELECT ?s ?sname ?p ?pname
WHERE { 
    wd:Q164750 ?s ?p  .
    # label
    ?p <http://schema.org/name> ?pname .
    ?s <http://schema.org/name> ?sname .
    FILTER regex(?pname, "university", "i") 
} 
limit 50
"""

print("Results")
run_query(queryString)

Results
[('s', 'http://www.wikidata.org/prop/direct/P166'), ('sname', 'award received'), ('p', 'http://www.wikidata.org/entity/Q42559576'), ('pname', 'honorary doctor of the University of Gothenburg')]
[('s', 'http://www.wikidata.org/prop/direct/P69'), ('sname', 'educated at'), ('p', 'http://www.wikidata.org/entity/Q2503794'), ('pname', 'Northumbria University')]


2

In [9]:
# show the university or degree of apple designers
queryString = """
SELECT ?university ?universityname
WHERE { 
    wd:Q164750 wdt:P69 ?university  .
    # label
    ?university <http://schema.org/name> ?universityname .
    #?s <http://schema.org/name> ?sname .
} 
limit 50
"""

print("Results")
run_query(queryString)

Results
[('university', 'http://www.wikidata.org/entity/Q2503794'), ('universityname', 'Northumbria University')]


1

#### Apple products and their designers

In [10]:
queryString = """
SELECT ?productname ?designername
WHERE { 
    ?product wdt:P178 wd:Q312 .
    ?product wdt:P287 ?designer .
    # label
    ?product <http://schema.org/name> ?productname .
    ?designer <http://schema.org/name> ?designername .
} 
ORDER BY ?productname
LIMIT 100
"""

print("Results")
run_query(queryString)

Results
[('productname', 'AirPods'), ('designername', 'Apple Inc.')]
[('productname', 'AirPods Max'), ('designername', 'Apple Inc.')]
[('productname', 'AirPods Pro'), ('designername', 'Apple Inc.')]
[('productname', 'AirPower'), ('designername', 'Apple Inc.')]
[('productname', 'Apple A series'), ('designername', 'Apple Inc.')]
[('productname', 'Apple A13 Bionic'), ('designername', 'Apple Inc.')]
[('productname', 'Apple A14 Bionic'), ('designername', 'Apple Inc.')]
[('productname', 'Apple A4'), ('designername', 'Apple Inc.')]
[('productname', 'Apple A5'), ('designername', 'Apple Inc.')]
[('productname', 'Apple A5X'), ('designername', 'Apple Inc.')]
[('productname', 'Apple A6'), ('designername', 'Apple Inc.')]
[('productname', 'Apple A6X'), ('designername', 'Apple Inc.')]
[('productname', 'Apple A7'), ('designername', 'Apple Inc.')]
[('productname', 'Apple A8'), ('designername', 'Apple Inc.')]
[('productname', 'Apple M series'), ('designername', 'Apple Inc.')]
[('productname', 'Apple M1'

43

In [11]:
# trademark `wd:Q167270`  | trademark     | node 
queryString = """
SELECT ?p ?pname ?o ?oname
WHERE { 

    wd:Q167270 ?p ?o .
    # Labels
    ?p <http://schema.org/name> ?pname .
    ?o <http://schema.org/name> ?oname .
    #FILTER regex(?pname, "company", "i") 
} 
LIMIT 20
"""

print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('pname', 'subclass of'), ('o', 'http://www.wikidata.org/entity/Q80071'), ('oname', 'symbol')]
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('pname', 'subclass of'), ('o', 'http://www.wikidata.org/entity/Q131257'), ('oname', 'intellectual property')]
[('p', 'http://www.wikidata.org/prop/direct/P1343'), ('pname', 'described by source'), ('o', 'http://www.wikidata.org/entity/Q602358'), ('oname', 'Brockhaus and Efron Encyclopedic Dictionary')]
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('pname', 'subclass of'), ('o', 'http://www.wikidata.org/entity/Q3695082'), ('oname', 'sign')]
[('p', 'http://www.wikidata.org/prop/direct/P913'), ('pname', 'notation'), ('o', 'http://www.wikidata.org/entity/Q1944044'), ('oname', 'trademark symbol')]
[('p', 'http://www.wikidata.org/prop/direct/P1424'), ('pname', "topic's main template"), ('o', 'http://www.wikidata.org/entity/Q5616238'), ('oname', 'Template:Infobox brand')]
[('p', 'h

10

In [12]:
# trademark `wd:Q167270`  | trademark     | node 
queryString = """
SELECT ?p ?pname ?o ?oname
WHERE { 

    ?p ?o wd:Q167270 .
    # Labels
    ?p <http://schema.org/name> ?pname .
    ?o <http://schema.org/name> ?oname .
    FILTER regex(?pname, "company", "i") 
} 
LIMIT 40
"""

print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/entity/Q7607050'), ('pname', "Stella D'oro Biscuit Company"), ('o', 'http://www.wikidata.org/prop/direct/P31'), ('oname', 'instance of')]
[('p', 'http://www.wikidata.org/entity/Q7910733'), ('pname', 'Valentin Blatz Brewing Company'), ('o', 'http://www.wikidata.org/prop/direct/P31'), ('oname', 'instance of')]
[('p', 'http://www.wikidata.org/entity/Q7624252'), ('pname', 'Stroh Brewery Company'), ('o', 'http://www.wikidata.org/prop/direct/P31'), ('oname', 'instance of')]
[('p', 'http://www.wikidata.org/entity/Q1686726'), ('pname', 'Jello Shoecompany'), ('o', 'http://www.wikidata.org/prop/direct/P31'), ('oname', 'instance of')]
[('p', 'http://www.wikidata.org/entity/Q6286885'), ('pname', 'Joseph Schlitz Brewing Company'), ('o', 'http://www.wikidata.org/prop/direct/P31'), ('oname', 'instance of')]
[('p', 'http://www.wikidata.org/entity/Q5577107'), ('pname', 'Goebel Brewing Company'), ('o', 'http://www.wikidata.org/prop/direct/P31'), ('oname', 'instanc

8

In [13]:
#`wd:Q312` | Apple Inc.    | node      |
queryString = """
SELECT ?p ?pname ?o ?oname
WHERE { 

    wd:Q312 ?p ?o .
    # Labels
    ?p <http://schema.org/name> ?pname .
    ?o <http://schema.org/name> ?oname .
    #FILTER regex(?pname, "company", "i") 
} 
LIMIT 20
"""

print("Results")
run_query(queryString)

## P1056

Results
[('p', 'http://www.wikidata.org/prop/direct/P1056'), ('pname', 'product or material produced'), ('o', 'http://www.wikidata.org/entity/Q26833946'), ('oname', 'iPhone\xa07 Plus')]
[('p', 'http://www.wikidata.org/prop/direct/P1056'), ('pname', 'product or material produced'), ('o', 'http://www.wikidata.org/entity/Q39510168'), ('oname', 'iPhone X')]
[('p', 'http://www.wikidata.org/prop/direct/P1056'), ('pname', 'product or material produced'), ('o', 'http://www.wikidata.org/entity/Q20962515'), ('oname', 'Apple Pencil')]
[('p', 'http://www.wikidata.org/prop/direct/P452'), ('pname', 'industry'), ('o', 'http://www.wikidata.org/entity/Q581105'), ('oname', 'consumer electronics')]
[('p', 'http://www.wikidata.org/prop/direct/P463'), ('pname', 'member of'), ('o', 'http://www.wikidata.org/entity/Q79100838'), ('oname', 'Zigbee Alliance')]
[('p', 'http://www.wikidata.org/prop/direct/P1056'), ('pname', 'product or material produced'), ('o', 'http://www.wikidata.org/entity/Q18010891'), ('oname

20

In [14]:
#`wd:Q312` | Apple Inc.    | node      |
queryString = """
SELECT ?p ?pname ?o ?oname
WHERE { 

    ?p ?o wd:Q312 .
    # Labels
    ?p <http://schema.org/name> ?pname .
    ?o <http://schema.org/name> ?oname .

} 
LIMIT 20
"""

print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/entity/Q98839064'), ('pname', 'Epic Games v. Apple'), ('o', 'http://www.wikidata.org/prop/direct/P1591'), ('oname', 'defendant')]
[('p', 'http://www.wikidata.org/entity/Q2995539'), ('pname', 'Apple Corps v Apple Computer'), ('o', 'http://www.wikidata.org/prop/direct/P1591'), ('oname', 'defendant')]
[('p', 'http://www.wikidata.org/entity/Q13218275'), ('pname', 'Apple logo'), ('o', 'http://www.wikidata.org/prop/direct/P1268'), ('oname', 'represents')]
[('p', 'http://www.wikidata.org/entity/Q3506674'), ('pname', 'Apple Switch ad campaign'), ('o', 'http://www.wikidata.org/prop/direct/P6291'), ('oname', 'advertises')]
[('p', 'http://www.wikidata.org/entity/Q420951'), ('pname', 'Get a Mac'), ('o', 'http://www.wikidata.org/prop/direct/P6291'), ('oname', 'advertises')]
[('p', 'http://www.wikidata.org/entity/Q1191023'), ('pname', 'Think different'), ('o', 'http://www.wikidata.org/prop/direct/P6291'), ('oname', 'advertises')]
[('p', 'http://www.wikidata.or

20

In [15]:
## Apple employer 
queryString = """
SELECT ?employer ?employername
WHERE { 

    ?employer wdt:P108 wd:Q312 .
    # Labels
    ?employer <http://schema.org/name> ?employername .
    #?o <http://schema.org/name> ?oname .
    #FILTER regex(?oname, "employer", "i") 
} 
LIMIT 20
"""

print("Results")
run_query(queryString)

Results
[('employer', 'http://www.wikidata.org/entity/Q16186567'), ('employername', 'Joe Ybarra')]
[('employer', 'http://www.wikidata.org/entity/Q200678'), ('employername', 'Queen Rania of Jordan')]
[('employer', 'http://www.wikidata.org/entity/Q21984549'), ('employername', 'Johny Srouji')]
[('employer', 'http://www.wikidata.org/entity/Q84197727'), ('employername', 'Te Hu')]
[('employer', 'http://www.wikidata.org/entity/Q55445014'), ('employername', 'Xinhao Wang')]
[('employer', 'http://www.wikidata.org/entity/Q19958103'), ('employername', 'Rod Holt')]
[('employer', 'http://www.wikidata.org/entity/Q16221584'), ('employername', 'Yoky Matsuoka')]
[('employer', 'http://www.wikidata.org/entity/Q98251'), ('employername', 'Hartmut Esslinger')]
[('employer', 'http://www.wikidata.org/entity/Q483382'), ('employername', 'Steve Wozniak')]
[('employer', 'http://www.wikidata.org/entity/Q282722'), ('employername', 'Trent Reznor')]
[('employer', 'http://www.wikidata.org/entity/Q529233'), ('employerna

20

In [16]:
# [('p', 'http://www.wikidata.org/entity/Q483382'), ('pname', 'Steve Wozniak')
queryString = """
SELECT ?p ?pname ?o ?oname
WHERE { 

    wd:Q483382 ?p ?o .
    # Labels
    ?p <http://schema.org/name> ?pname .
    ?o <http://schema.org/name> ?oname .

} 
LIMIT 40
"""

print("Results")
run_query(queryString)

#`wdt:P31`     | instance of   | predicate
#[('p', 'http://www.wikidata.org/prop/direct/P31'), ('pname', 'instance of'),
#('o', 'http://www.wikidata.org/entity/Q5'), ('oname', 'human')]

Results
[('p', 'http://www.wikidata.org/prop/direct/P463'), ('pname', 'member of'), ('o', 'http://www.wikidata.org/entity/Q159452'), ('oname', 'capoeira')]
[('p', 'http://www.wikidata.org/prop/direct/P69'), ('pname', 'educated at'), ('o', 'http://www.wikidata.org/entity/Q736674'), ('oname', 'University of Colorado Boulder')]
[('p', 'http://www.wikidata.org/prop/direct/P166'), ('pname', 'award received'), ('o', 'http://www.wikidata.org/entity/Q1541081'), ('oname', 'Grace Murray Hopper Award')]
[('p', 'http://www.wikidata.org/prop/direct/P166'), ('pname', 'award received'), ('o', 'http://www.wikidata.org/entity/Q29545180'), ('oname', 'Computer History Museum fellow')]
[('p', 'http://www.wikidata.org/prop/direct/P101'), ('pname', 'field of work'), ('o', 'http://www.wikidata.org/entity/Q80993'), ('oname', 'software engineering')]
[('p', 'http://www.wikidata.org/prop/direct/P800'), ('pname', 'notable work'), ('o', 'http://www.wikidata.org/entity/Q177491'), ('oname', 'Breakout')]
[('p', 'htt

40

In [17]:
# [('p', 'http://www.wikidata.org/entity/Q483382'), ('pname', 'Steve Wozniak')
queryString = """
SELECT ?human ?humanname
WHERE { 

    ?human wdt:P31 wd:Q5 .
    # Labels

    ?human <http://schema.org/name> ?humanname .

} 
LIMIT 40
"""

print("Results")
run_query(queryString)

#`wdt:P31`     | instance of   | predicate
#[('p', 'http://www.wikidata.org/prop/direct/P31'), ('pname', 'instance of'),
#('o', 'http://www.wikidata.org/entity/Q5'), ('oname', 'human')]

Results
[('human', 'http://www.wikidata.org/entity/Q1002728'), ('humanname', 'Fonsi Nieto')]
[('human', 'http://www.wikidata.org/entity/Q100343342'), ('humanname', 'Lubomír Stoklásek')]
[('human', 'http://www.wikidata.org/entity/Q102115260'), ('humanname', 'Kojiro Kobayashi')]
[('human', 'http://www.wikidata.org/entity/Q102118153'), ('humanname', 'Miguel E. M. Herrera')]
[('human', 'http://www.wikidata.org/entity/Q105753880'), ('humanname', 'Mateo Egea')]
[('human', 'http://www.wikidata.org/entity/Q107520918'), ('humanname', 'Jerónimo Latorre')]
[('human', 'http://www.wikidata.org/entity/Q11404285'), ('humanname', 'Mikiko Kitano')]
[('human', 'http://www.wikidata.org/entity/Q11520274'), ('humanname', 'Honjō Fusanaga')]
[('human', 'http://www.wikidata.org/entity/Q11562694'), ('humanname', 'Takayoshi Watanabe')]
[('human', 'http://www.wikidata.org/entity/Q11685026'), ('humanname', 'Joan de Gènova')]
[('human', 'http://www.wikidata.org/entity/Q11686949'), ('humanname', 'José Ximénez')]
[(

40

In [18]:
# [('p', 'http://www.wikidata.org/entity/Q483382'), ('pname', 'Steve Wozniak')
queryString = """
SELECT ?p ?pname ?o ?oname
WHERE { 

    wd:Q483382 ?p ?o .
    # Labels
    ?p <http://schema.org/name> ?pname .
    ?o <http://schema.org/name> ?oname .
    FILTER regex(?oname, "computer", "i") 
} 
LIMIT 40
"""

print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P106'), ('pname', 'occupation'), ('o', 'http://www.wikidata.org/entity/Q82594'), ('oname', 'computer scientist')]
[('p', 'http://www.wikidata.org/prop/direct/P1343'), ('pname', 'described by source'), ('o', 'http://www.wikidata.org/entity/Q1432544'), ('oname', 'Hackers: Heroes of the Computer Revolution')]
[('p', 'http://www.wikidata.org/prop/direct/P166'), ('pname', 'award received'), ('o', 'http://www.wikidata.org/entity/Q29545180'), ('oname', 'Computer History Museum fellow')]


3

In [19]:
# [('p', 'http://www.wikidata.org/entity/Q483382'), ('pname', 'Steve Wozniak')
queryString = """
SELECT ?p ?pname ?o ?oname
WHERE { 

    wd:Q483382 ?p ?o .
    # Labels
    ?p <http://schema.org/name> ?pname .
    ?o <http://schema.org/name> ?oname .

    FILTER regex(?pname, "academic", "i") 
} 
LIMIT 20
"""

print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P512'), ('pname', 'academic degree'), ('o', 'http://www.wikidata.org/entity/Q63107776'), ('oname', 'Bachelor of Science in Engineering')]


1

In [20]:
# [('p', 'http://www.wikidata.org/entity/Q483382'), ('pname', 'Steve Wozniak')
queryString = """
SELECT ?o ?oname ?universityname
WHERE { 

    wd:Q483382 wdt:P512 ?o .
    wd:Q483382 wdt:P69 ?university  .
    # label
    ?university <http://schema.org/name> ?universityname .

    ?o <http://schema.org/name> ?oname .

} 
LIMIT 20
"""

print("Results")
run_query(queryString)

Results
[('o', 'http://www.wikidata.org/entity/Q63107776'), ('oname', 'Bachelor of Science in Engineering'), ('universityname', 'University of Colorado Boulder')]
[('o', 'http://www.wikidata.org/entity/Q63107776'), ('oname', 'Bachelor of Science in Engineering'), ('universityname', 'De Anza College')]
[('o', 'http://www.wikidata.org/entity/Q63107776'), ('oname', 'Bachelor of Science in Engineering'), ('universityname', 'University of California, Berkeley')]
[('o', 'http://www.wikidata.org/entity/Q63107776'), ('oname', 'Bachelor of Science in Engineering'), ('universityname', 'Homestead High School')]
[('o', 'http://www.wikidata.org/entity/Q63107776'), ('oname', 'Bachelor of Science in Engineering'), ('universityname', 'University of Colorado')]


5

## Show the employers of Apple and the institutions they have studied and their level of studies. 

In [21]:
queryString = """
SELECT ?employername
(GROUP_CONCAT(DISTINCT ?degreename; SEPARATOR=", ") AS ?degreenames) 
(GROUP_CONCAT(DISTINCT ?universityname; SEPARATOR=", ") AS ?universitynames)
WHERE { 
    ?employer wdt:P108 wd:Q312 .
    
    #?employer wdt:P512 ?degree .
    ?employer wdt:P69 ?university  .
    OPTIONAL { ?employer wdt:P512 ?degree }
    # label
    ?university <http://schema.org/name> ?universityname .
    ?employer <http://schema.org/name> ?employername .
    ?degree <http://schema.org/name> ?degreename .
    
} 
ORDER BY ?employername 
LIMIT 20
"""

print("Results")
run_query(queryString)

Results
[('employername', 'Arthur D. Levinson'), ('degreenames', 'Doctor of Philosophy'), ('universitynames', 'Princeton University, University of California, San Francisco, University of Washington')]
[('employername', 'Bertrand Serlet'), ('degreenames', 'Doctor of Philosophy'), ('universitynames', 'University of Paris-Sud')]
[('employername', 'Eric Schmidt'), ('degreenames', 'doctorate'), ('universitynames', 'Princeton University, University of California, Berkeley, Yorktown High School')]
[('employername', 'Peter Dewald'), ('degreenames', 'Master of Science'), ('universitynames', 'Leibniz University Hannover')]
[('employername', 'Steve Wozniak'), ('degreenames', 'Bachelor of Science in Engineering'), ('universitynames', 'De Anza College, Homestead High School, University of California, Berkeley, University of Colorado, University of Colorado Boulder')]
[('employername', 'Suki Lee'), ('degreenames', "bachelor's degree"), ('universitynames', 'San José State University')]
[('employerna

9

In [22]:
queryString = """
SELECT ?employername ?universityname
WHERE { 
    ?employer wdt:P108 wd:Q312 .
    ?employer wdt:P69 ?university  .

    # label
    ?university <http://schema.org/name> ?universityname .
    ?employer <http://schema.org/name> ?employername .

    
} 
ORDER BY ?employername 
LIMIT 30
"""

print("Results")
run_query(queryString)

Results
[('employername', 'Adam Cheyer'), ('universityname', 'University of California, Los Angeles')]
[('employername', 'Adam Cheyer'), ('universityname', 'Brandeis University')]
[('employername', 'Adam Cheyer'), ('universityname', 'Sharon High School')]
[('employername', 'Alan B. Oppenheimer'), ('universityname', 'Massachusetts Institute of Technology')]
[('employername', 'Alex Colburn'), ('universityname', 'University of Washington')]
[('employername', 'Alice K. Hartley'), ('universityname', 'Massachusetts Institute of Technology')]
[('employername', 'Allen Kenneth Briggs'), ('universityname', 'Virginia Tech')]
[('employername', 'Allen Kenneth Briggs'), ('universityname', 'North Carolina School of Science and Mathematics')]
[('employername', 'Amelia Boone'), ('universityname', 'Washington University in St.\xa0Louis')]
[('employername', 'Amelia Boone'), ('universityname', 'University of Washington School of Law')]
[('employername', 'Anand Lal Shimpi'), ('universityname', 'William G. 

30

## Show a Trademark employers and the name of its industry.

In [23]:
#the BGP to retrieve all entities that are trademarks
queryString = """
SELECT ?trademarkname
WHERE { 

?trademark wdt:P31/wdt:P279* wd:Q167270  . 
?trademark <http://schema.org/name> ?trademarkname .

} 
LIMIT 20
"""

print("Results")
run_query(queryString)

Results
[('trademarkname', 'Michelob')]
[('trademarkname', 'Bernard')]
[('trademarkname', 'TOP Media')]
[('trademarkname', 'TOP Media')]
[('trademarkname', 'Malco Theatres')]
[('trademarkname', 'Malco Theatres')]
[('trademarkname', 'MovieTime Cinemas')]
[('trademarkname', 'MovieTime Cinemas')]
[('trademarkname', 'B2M Entertainment')]
[('trademarkname', 'B2M Entertainment')]
[('trademarkname', 'Cathay Organisation')]
[('trademarkname', 'Cathay Organisation')]
[('trademarkname', 'Albrecht')]
[('trademarkname', 'Network NorthWest')]
[('trademarkname', 'H2 Media Entertainment')]
[('trademarkname', 'H2 Media Entertainment')]
[('trademarkname', 'N.A.P.Entertainment')]
[('trademarkname', 'N.A.P.Entertainment')]
[('trademarkname', 'Vue Entertainment')]
[('trademarkname', 'Vue Entertainment')]


20

In [24]:

queryString = """
SELECT ?trademarkname ?industryname 
(GROUP_CONCAT(DISTINCT ?employername; SEPARATOR=", ") AS ?employernames)
WHERE { 
    
    ?trademark wdt:P31/wdt:P279* wd:Q167270  . 
    ?trademark wdt:P452 ?industry .
    ?employer wdt:P108 ?trademark .
    # Labels
    ?employer <http://schema.org/name> ?employername .
    ?trademark <http://schema.org/name> ?trademarkname .
    ?industry <http://schema.org/name> ?industryname .
    #FILTER regex(?industryname, "electronics", "i") 
} 
ORDER BY ?industryname
LIMIT 10
"""

print("Results")
run_query(queryString)

Results
[('trademarkname', 'McDonald’s'), ('industryname', 'Accommodation and food service activities'), ('employernames', 'Barbara Meier, Cécile Jolly, Dan Coudreaut, Dieter Kempf, Jerome Tafani, Jim Delligatti, Lauren Boebert, Patrick Mispolet, Ray Kroc, Steve Easterbrook')]
[('trademarkname', 'Crédit Lyonnais'), ('industryname', 'Financial service activities, except insurance and pension funding'), ('employernames', 'AJ Mediratta, Claire Houry, Claude Villers, Daniel Maisonnave, Georges Richou, Jacques Letourneur, Janette Rutterford, Jean Boussemart, Jean-Paul Mazoyer, Loïg Chesnais-Girard, Nigel Farage, Vladimír Valach')]
[('trademarkname', 'Calvé'), ('industryname', 'Financial service activities, except insurance and pension funding'), ('employernames', 'Mark Rutte')]
[('trademarkname', 'Dresdner Bank'), ('industryname', 'Financial service activities, except insurance and pension funding'), ('employernames', 'Angelika Kausche, Bruce Wasserstein, Eduardo Díez-Hochleitner, Frank Nop

10

In [25]:

queryString = """
SELECT ?trademarkname 
(GROUP_CONCAT(DISTINCT ?employername; SEPARATOR=", ") AS ?employernames)
?industryname 
WHERE { 
    
    ?trademark wdt:P31/wdt:P279* wd:Q167270  . 
    ?trademark wdt:P452 ?industry .
    ?employer wdt:P108 ?trademark .
    # Labels
    ?employer <http://schema.org/name> ?employername .
    ?trademark <http://schema.org/name> ?trademarkname .
    ?industry <http://schema.org/name> ?industryname .
    FILTER regex(?industryname, "electronics", "i") 
} 
ORDER BY ?trademarkname
LIMIT 40
"""

print("Results")
run_query(queryString)

Results
[('trademarkname', 'Apple Inc.'), ('employernames', "Adam Cheyer, Alan B. Oppenheimer, Alex Colburn, Alex Fielding, Alex Hitzinger, Alice K. Hartley, Allen Kenneth Briggs, Amelia Boone, Amir Alavi, Amir Tahmasbi, Anand Lal Shimpi, Andrea Jung, Andy Hertzfeld, Andy Rubin, Angela Ahrendts, Ann Schmeltz Bowers, Arthur D. Levinson, Ashish Shrivastava, Avie Tevanian, Bertrand Serlet, Bill Atkinson, Bill Fernandez, Bill Kincaid, Bob Belleville, Bob Mansfield, Bret Victor, Brit Morin, Bruce Daniels, Bruce Horn, Bruce Sewell, Bruce Tognazzini, Bud Tribble, Burrell Smith, C. Lloyd Mahaffey, Caitlin Kalinowski, Cathy Edwards, Chris Espinosa, Chris R Marcellino, Chrisann Brennan, Christoph Paasch, Christopher Stringer, Clement Mok, Craig Federighi, Craig H Mermel, Cynthia Solomon, Dale Fuller, Dan Ingalls, Dan Riccio, Daniel Kottke, Darin Adler, Dave Hyatt, Dave Morin, David Hwang, David Moon, David Nagel, Dawei Lu, Deirdre O’Brien, Del Yocam, Dominic Giampaolo, Donald Norman, Donna Augus

9

## Count the number of employers of a trademark 

In [26]:
queryString = """
SELECT ?trademarkname (COUNT (?employer) AS ?countemployer)
WHERE { 
    ?trademark wdt:P31/wdt:P279* wd:Q167270  . 
    ?trademark wdt:P452 ?industry .
    ?employer wdt:P108 ?trademark .
    # Labels
    ?employer <http://schema.org/name> ?employername .
    ?trademark <http://schema.org/name> ?trademarkname .
    #?industry <http://schema.org/name> ?industryname .
    #FILTER regex(?industryname, "electronics", "i") 
} 
ORDER BY DESC (?countemployer)
LIMIT 20
"""

print("Results")
run_query(queryString)

Results
[('trademarkname', 'Apple Inc.'), ('countemployer', '2736')]
[('trademarkname', 'Nokia'), ('countemployer', '222')]
[('trademarkname', 'Walmart'), ('countemployer', '128')]
[('trademarkname', 'EITB Euskal Irrati Telebista'), ('countemployer', '126')]
[('trademarkname', 'McDonald’s'), ('countemployer', '120')]
[('trademarkname', 'Sony Music'), ('countemployer', '78')]
[('trademarkname', 'Polskie Radio'), ('countemployer', '66')]
[('trademarkname', 'Boots'), ('countemployer', '66')]
[('trademarkname', 'Nestlé'), ('countemployer', '62')]
[('trademarkname', '3M'), ('countemployer', '59')]
[('trademarkname', 'BP'), ('countemployer', '58')]
[('trademarkname', 'Ford Motor Company'), ('countemployer', '57')]
[('trademarkname', 'EMI'), ('countemployer', '56')]
[('trademarkname', 'Pathé'), ('countemployer', '52')]
[('trademarkname', 'Nike'), ('countemployer', '42')]
[('trademarkname', 'Tesla, Inc.'), ('countemployer', '40')]
[('trademarkname', 'Tiffany & Co.'), ('countemployer', '36')]
[

20

## Identify the BGP for the person related to a given trademark

In [27]:
queryString = """
SELECT ?trademarkname
(GROUP_CONCAT(DISTINCT ?humanname; SEPARATOR=", ") AS ?humannames)

WHERE { 
    ?human ?relation ?trademark .
    ?trademark wdt:P31/wdt:P279* wd:Q167270  . 
   #?trademark wdt:P452 ?industry .
    #?employer wdt:P108 ?trademark .
    ?human wdt:P31 wd:Q5 .
    # Labels
    ?human <http://schema.org/name> ?humanname .
    ?relation <http://schema.org/name> ?relationname .
    #?employer <http://schema.org/name> ?employername .
    ?trademark <http://schema.org/name> ?trademarkname .
    #?industry <http://schema.org/name> ?industryname .
   #FILTER regex(?industryname, "electronics", "i") 
} 

LIMIT 40
"""

print("Results")
run_query(queryString)

Results
[('trademarkname', 'Audio Fidelity'), ('humannames', 'Al Hirt, Don Shirley, Eddie Cantor, Elmo Hope, Johnny Puleo, Lalo Schifrin, Larry Adler, Leon Berry, Lionel Hampton, Louis Armstrong, Louisa Jo Killen, Mohammed El-Bakkar, Myron Cohen, Oscar Brand, Patachou')]
[('trademarkname', 'Wing Records'), ('humannames', 'Vanessa Williams')]
[('trademarkname', 'Crest Records'), ('humannames', 'Bobby Edwards, Carol Kaye, Donna Loren, Eddie Cochran, Eileen Barton, Glen Campbell, Jean Chapel, Jerry Capehart, Jewel Akens, Jimmy Bowen, Phil Baugh, Terry Fell, Tom Tall')]
[('trademarkname', 'MUN HWA IN'), ('humannames', 'Youra')]
[('trademarkname', 'Minaret'), ('humannames', 'Finley Duncan, Herb Shucher')]
[('trademarkname', 'Kitchenware Records'), ('humannames', 'Chris Urbanowicz, Kate Walsh, Russell Leetch, Tom Smith')]
[('trademarkname', 'Doublemint'), ('humannames', 'Tamera Mowry')]
[('trademarkname', 'Rise Records'), ('humannames', 'Danny Thompson, Derek Bloom, Jerry Roush, Jon Kintz, J

40

In [28]:
queryString = """
SELECT ?humanname ?relationname ?trademarkname
WHERE { 
    ?human ?relation ?trademark .
    ?trademark wdt:P31/wdt:P279* wd:Q167270  . 
   #?trademark wdt:P452 ?industry .
    #?employer wdt:P108 ?trademark .
    ?human wdt:P31 wd:Q5 .
    # Labels
    ?human <http://schema.org/name> ?humanname .
    ?relation <http://schema.org/name> ?relationname .
    #?employer <http://schema.org/name> ?employername .
    ?trademark <http://schema.org/name> ?trademarkname .
    #?industry <http://schema.org/name> ?industryname .
   #FILTER regex(?industryname, "electronics", "i") 
} 

LIMIT 40
"""

print("Results")
run_query(queryString)

Results
[('humanname', 'Roxana Ortega'), ('relationname', 'described by source'), ('trademarkname', 'Cineplex')]
[('humanname', 'Roxana Ortega'), ('relationname', 'described by source'), ('trademarkname', 'Cineplex')]
[('humanname', 'Yūya Yagira'), ('relationname', 'employer'), ('trademarkname', 'Stardust Promotion')]
[('humanname', 'Yūya Yagira'), ('relationname', 'employer'), ('trademarkname', 'Stardust Promotion')]
[('humanname', 'Ovidio G. Assonitis'), ('relationname', 'employer'), ('trademarkname', 'The Cannon Group')]
[('humanname', 'Ovidio G. Assonitis'), ('relationname', 'employer'), ('trademarkname', 'The Cannon Group')]
[('humanname', 'Yang Hyun-seok'), ('relationname', 'employer'), ('trademarkname', 'YG Entertainment')]
[('humanname', 'Yang Hyun-seok'), ('relationname', 'employer'), ('trademarkname', 'YG Entertainment')]
[('humanname', 'Queen Anne of Romania'), ('relationname', 'employer'), ('trademarkname', "Macy's")]
[('humanname', 'Queen Anne of Romania'), ('relationname'

40

In [29]:
#`wd:Q312` | Apple Inc.    | node      |
queryString = """
SELECT ?p ?pname ?o ?oname
WHERE { 

    wd:Q312 ?p ?o .
    # Labels
    ?p <http://schema.org/name> ?pname .
    ?o <http://schema.org/name> ?oname .
    FILTER regex(?oname, "company", "i") 
} 
LIMIT 20
"""

print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('pname', 'instance of'), ('o', 'http://www.wikidata.org/entity/Q891723'), ('oname', 'public company')]


1

In [30]:
#`wd:Q312` | Apple Inc.    | node      |
queryString = """
SELECT ?p ?pname ?o ?oname
WHERE { 

    ?p ?o wd:Q312 .
    # Labels
    ?p <http://schema.org/name> ?pname .
    ?o <http://schema.org/name> ?oname .
    FILTER regex(?pname, "company", "i") 
} 
LIMIT 20
"""

print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/entity/P3875'), ('pname', 'Justia Patents company ID'), ('o', 'http://www.wikidata.org/prop/direct/P1855'), ('oname', 'Wikidata property example')]
[('p', 'http://www.wikidata.org/entity/P7283'), ('pname', 'Spectrum Management System Company Number'), ('o', 'http://www.wikidata.org/prop/direct/P1855'), ('oname', 'Wikidata property example')]


2

In [31]:
#`wd:Q312` | Apple Inc.    | node      |
queryString = """
SELECT ?group ?groupname ?relation ?relationname
WHERE { 

    ?group ?relation wd:Q312 .
    # Labels
    ?group <http://schema.org/name> ?groupname .
    ?relation <http://schema.org/name> ?relationname .
    FILTER regex(?groupname, "group", "i") 
} 
LIMIT 20
"""

print("Results")
run_query(queryString)

Results
[('group', 'http://www.wikidata.org/entity/Q4020840'), ('groupname', 'Workgroup Manager'), ('relation', 'http://www.wikidata.org/prop/direct/P178'), ('relationname', 'developer')]
[('group', 'http://www.wikidata.org/entity/Q849363'), ('groupname', 'The Vanguard Group'), ('relation', 'http://www.wikidata.org/prop/direct/P1830'), ('relationname', 'owner of')]
[('group', 'http://www.wikidata.org/entity/Q4781185'), ('groupname', 'Apple Industrial Design Group'), ('relation', 'http://www.wikidata.org/prop/direct/P361'), ('relationname', 'part of')]


3

In [32]:
#`wd:Q312` | Apple Inc.    | node      |
queryString = """
SELECT ?p ?pname ?o ?oname
WHERE { 

    wd:Q312 ?p ?o .
    # Labels
    ?p <http://schema.org/name> ?pname .
    ?o <http://schema.org/name> ?oname .
    FILTER regex(?oname, "group", "i") 
} 
LIMIT 20
"""

print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P127'), ('pname', 'owned by'), ('o', 'http://www.wikidata.org/entity/Q849363'), ('oname', 'The Vanguard Group')]


1

In [33]:
#`wd:Q312` | Apple Inc.    | node      |
queryString = """
SELECT ?p ?pname ?o ?oname
WHERE { 

    ?p ?o wd:Q4781185 .
    # Labels
    ?p <http://schema.org/name> ?pname .
    ?o <http://schema.org/name> ?oname .
    #FILTER regex(?oname, "group", "i") 
} 
LIMIT 20
"""

print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/entity/Q4034223'), ('pname', 'Apple Pro Mouse'), ('o', 'http://www.wikidata.org/prop/direct/P170'), ('oname', 'creator')]
[('p', 'http://www.wikidata.org/entity/Q65037131'), ('pname', 'Evans Hankey'), ('o', 'http://www.wikidata.org/prop/direct/P463'), ('oname', 'member of')]


2

In [34]:
# different companies related to Apple 

In [35]:
#`wd:Q312` | Apple Inc.    | node      |
queryString = """
SELECT ?group ?groupname ?relation ?relationname
WHERE { 

    ?group ?relation wd:Q312 .
    # Labels
    ?group <http://schema.org/name> ?groupname .
    ?relation <http://schema.org/name> ?relationname .
    FILTER regex(?groupname, "group", "i") 
} 
LIMIT 20
"""

print("Results")
run_query(queryString)

Results
[('group', 'http://www.wikidata.org/entity/Q4020840'), ('groupname', 'Workgroup Manager'), ('relation', 'http://www.wikidata.org/prop/direct/P178'), ('relationname', 'developer')]
[('group', 'http://www.wikidata.org/entity/Q849363'), ('groupname', 'The Vanguard Group'), ('relation', 'http://www.wikidata.org/prop/direct/P1830'), ('relationname', 'owner of')]
[('group', 'http://www.wikidata.org/entity/Q4781185'), ('groupname', 'Apple Industrial Design Group'), ('relation', 'http://www.wikidata.org/prop/direct/P361'), ('relationname', 'part of')]


3

## Identify the BGP for the company related to a given trademark

In [36]:
queryString = """
SELECT DISTINCT ?trademarkname ?relationnames?groupname
WHERE { 

    ?group ?relation ?trademark .
    ?trademark wdt:P31/wdt:P279* wd:Q167270  . 
    #?trademark wdt:P452 ?industry .
    #?employer wdt:P108 ?trademark .
    # Labels
    #?employer <http://schema.org/name> ?employername .
    ?trademark <http://schema.org/name> ?trademarkname .
    #?industry <http://schema.org/name> ?industryname .
    ?group <http://schema.org/name> ?groupname .
    ?relation <http://schema.org/name> ?relationname .
    FILTER regex(?groupname, "group", "i") 
} 
ORDER BY ?trademarkname
LIMIT 20
"""

print("Results")
run_query(queryString)

Results
[('trademarkname', '960 Music Group'), ('groupname', '960 Music Group')]
[('trademarkname', 'A&M Records'), ('groupname', 'November Group')]
[('trademarkname', 'AB Groupe'), ('groupname', 'Category:AB Groupe')]
[('trademarkname', 'ABC'), ('groupname', 'S Group')]
[('trademarkname', 'ACT Music'), ('groupname', 'The Real Group')]
[('trademarkname', 'AEON MALL'), ('groupname', 'Shopping centers of AEON Group')]
[('trademarkname', 'AMC Theatres'), ('groupname', 'Odeon Cinemas Group')]
[('trademarkname', 'AMC Theatres'), ('groupname', 'Nordic Cinema Group')]
[('trademarkname', 'AMC Theatres'), ('groupname', 'Wanda Group')]
[('trademarkname', 'AWAL'), ('groupname', 'E2A Music Group, LLC')]
[('trademarkname', 'Absolut Vodka'), ('groupname', 'V&S Group')]
[('trademarkname', 'Academic Press'), ('groupname', 'RELX Group')]
[('trademarkname', 'Alabianca Records'), ('groupname', 'Ala Bianca Group S.r.l.')]
[('trademarkname', 'Alarma Records'), ('groupname', 'Meis Music Group')]
[('trademar

20

In [37]:
queryString = """
SELECT DISTINCT ?trademarkname 
(GROUP_CONCAT(DISTINCT ?relationname; SEPARATOR=", ") AS ?relationnames)
(GROUP_CONCAT(DISTINCT ?groupname; SEPARATOR=", ") AS ?groupnames)
WHERE { 

    ?group ?relation ?trademark .
    ?trademark wdt:P31/wdt:P279* wd:Q167270  . 
    #?trademark wdt:P452 ?industry .
    #?employer wdt:P108 ?trademark .
    # Labels
    #?employer <http://schema.org/name> ?employername .
    ?trademark <http://schema.org/name> ?trademarkname .
    #?industry <http://schema.org/name> ?industryname .
    ?group <http://schema.org/name> ?groupname .
    ?relation <http://schema.org/name> ?relationname .
    FILTER regex(?groupname, "group", "i") 
} 
ORDER BY ?trademarkname
LIMIT 20
"""

print("Results")
run_query(queryString)

Results
[('trademarkname', '960 Music Group'), ('relationnames', 'parent organization, subsidiary'), ('groupnames', '960 Music Group')]
[('trademarkname', 'A&M Records'), ('relationnames', 'record label'), ('groupnames', 'November Group')]
[('trademarkname', 'AB Groupe'), ('relationnames', "category's main topic"), ('groupnames', 'Category:AB Groupe')]
[('trademarkname', 'ABC'), ('relationnames', 'subsidiary'), ('groupnames', 'S Group')]
[('trademarkname', 'ACT Music'), ('relationnames', 'record label'), ('groupnames', 'The Real Group')]
[('trademarkname', 'AEON MALL'), ('relationnames', 'operator'), ('groupnames', 'Shopping centers of AEON Group')]
[('trademarkname', 'AMC Theatres'), ('relationnames', 'founded by, owned by, subsidiary'), ('groupnames', 'Nordic Cinema Group, Odeon Cinemas Group, Wanda Group')]
[('trademarkname', 'AWAL'), ('relationnames', 'distributed by'), ('groupnames', 'E2A Music Group, LLC')]
[('trademarkname', 'Absolut Vodka'), ('relationnames', 'product or materi

20

In [38]:
# electronic trademarks
queryString = """
SELECT DISTINCT ?trademarkname ?relationname ?groupname
WHERE { 

    ?group ?relation ?trademark .
    ?trademark wdt:P31/wdt:P279* wd:Q167270  . 
    ?trademark wdt:P452 ?industry .
    #?employer wdt:P108 ?trademark .
    # Labels
    #?employer <http://schema.org/name> ?employername .
    ?trademark <http://schema.org/name> ?trademarkname .
    ?industry <http://schema.org/name> ?industryname .
    ?group <http://schema.org/name> ?groupname .
    ?relation <http://schema.org/name> ?relationname .
    FILTER regex(?groupname, "group", "i") 
    FILTER regex(?industryname, "electronic", "i") 
} 
ORDER BY ?trademarkname
LIMIT 20
"""

print("Results")
run_query(queryString)

Results
[('trademarkname', 'Apple Inc.'), ('relationname', 'developer'), ('groupname', 'Workgroup Manager')]
[('trademarkname', 'Apple Inc.'), ('relationname', 'owner of'), ('groupname', 'The Vanguard Group')]
[('trademarkname', 'Apple Inc.'), ('relationname', 'part of'), ('groupname', 'Apple Industrial Design Group')]
[('trademarkname', 'EMI'), ('relationname', 'founded by'), ('groupname', 'Capitol Music Group')]
[('trademarkname', 'EMI'), ('relationname', 'subsidiary'), ('groupname', 'Warner Music Group')]
[('trademarkname', 'EMI'), ('relationname', 'subsidiary'), ('groupname', 'Disney Music Group')]
[('trademarkname', 'EMI'), ('relationname', 'said to be the same as'), ('groupname', 'EMI Group Limited')]
[('trademarkname', 'EMI'), ('relationname', 'category combines topics'), ('groupname', 'Category:EMI Group artists')]


8

In [39]:
# food trademarks
queryString = """
SELECT DISTINCT ?trademarkname ?relationname ?groupname
WHERE { 

    ?group ?relation ?trademark .
    ?trademark wdt:P31/wdt:P279* wd:Q167270  . 
    ?trademark wdt:P452 ?industry .
    #?employer wdt:P108 ?trademark .
    # Labels
    #?employer <http://schema.org/name> ?employername .
    ?trademark <http://schema.org/name> ?trademarkname .
    ?industry <http://schema.org/name> ?industryname .
    ?group <http://schema.org/name> ?groupname .
    ?relation <http://schema.org/name> ?relationname .
    FILTER regex(?groupname, "group", "i") 
    FILTER regex(?industryname, "food", "i") 
} 
ORDER BY ?trademarkname
LIMIT 20
"""

print("Results")
run_query(queryString)

Results
[('trademarkname', 'Bread Société'), ('relationname', 'subsidiary'), ('groupname', 'Ting Hsin International Group')]
[('trademarkname', "Domino's Pizza"), ('relationname', 'parent organization'), ('groupname', "Domino's Pizza Group plc")]
[('trademarkname', 'Mr. Sub'), ('relationname', 'subsidiary'), ('groupname', 'MTY Food Group')]
[('trademarkname', 'Paris Croissant'), ('relationname', 'owner of'), ('groupname', 'SPC Group')]
[('trademarkname', 'Pollo Tropical'), ('relationname', 'subsidiary'), ('groupname', 'Fiesta Restaurant Group')]
[('trademarkname', 'REWE Group'), ('relationname', "category's main topic"), ('groupname', 'Category:Rewe Group')]
[('trademarkname', 'REWE Group'), ('relationname', 'parent organization'), ('groupname', 'DER Touristik Group')]
[('trademarkname', 'Sushi Shop'), ('relationname', 'subsidiary'), ('groupname', 'MTY Food Group')]
[('trademarkname', 'The Pizza Company'), ('relationname', 'subsidiary'), ('groupname', 'Minor Food Group')]
[('trademarkn

10

In [40]:
# entertainment trademarks
queryString = """
SELECT DISTINCT ?trademarkname ?relationname ?groupname
WHERE { 

    ?group ?relation ?trademark .
    ?trademark wdt:P31/wdt:P279* wd:Q167270  . 
    ?trademark wdt:P452 ?industry .
    #?employer wdt:P108 ?trademark .
    # Labels
    #?employer <http://schema.org/name> ?employername .
    ?trademark <http://schema.org/name> ?trademarkname .
    ?industry <http://schema.org/name> ?industryname .
    ?group <http://schema.org/name> ?groupname .
    ?relation <http://schema.org/name> ?relationname .
    FILTER regex(?groupname, "group", "i") 
    FILTER regex(?industryname, "entertainment", "i") 
} 
ORDER BY ?trademarkname
LIMIT 20
"""

print("Results")
run_query(queryString)

Results
[('trademarkname', '960 Music Group'), ('relationname', 'subsidiary'), ('groupname', '960 Music Group')]
[('trademarkname', '960 Music Group'), ('relationname', 'parent organization'), ('groupname', '960 Music Group')]
[('trademarkname', 'AMC Theatres'), ('relationname', 'founded by'), ('groupname', 'Odeon Cinemas Group')]
[('trademarkname', 'AMC Theatres'), ('relationname', 'owned by'), ('groupname', 'Odeon Cinemas Group')]
[('trademarkname', 'AMC Theatres'), ('relationname', 'owned by'), ('groupname', 'Nordic Cinema Group')]
[('trademarkname', 'AMC Theatres'), ('relationname', 'subsidiary'), ('groupname', 'Wanda Group')]
[('trademarkname', 'Máximo Records'), ('relationname', 'subsidiary'), ('groupname', 'Maximo Group Network')]
[('trademarkname', 'National Amusements'), ('relationname', 'owned by'), ('groupname', 'Paramount Stations Group')]
[('trademarkname', 'Rotana Group'), ('relationname', "category's main topic"), ('groupname', 'Category:Rotana Group')]
[('trademarkname'

18

## Identify the BGP to retrieve types or categories for a given trademark

In [41]:
# categories or types
queryString = """
SELECT ?trademarkname (GROUP_CONCAT(DISTINCT ?categoryname; SEPARATOR=", ") AS ?categorynames) 
WHERE { 
    
    ?trademark wdt:P31/wdt:P279* wd:Q167270  . 
    ?trademark wdt:P452 ?category .
    # Labels
    ?trademark <http://schema.org/name> ?trademarkname .
    ?category <http://schema.org/name> ?categoryname .

} 
ORDER BY ?trademarkname
LIMIT 40
"""

print("Results")
run_query(queryString)

Results
[('trademarkname', '%Arabica'), ('categorynames', 'coffee')]
[('trademarkname', '01055'), ('categorynames', 'telecommunication')]
[('trademarkname', '100PROmotion'), ('categorynames', 'music industry')]
[('trademarkname', '101 Drogerie'), ('categorynames', 'drugstore')]
[('trademarkname', '1333 Entertainment'), ('categorynames', 'music industry')]
[('trademarkname', '2-Tone Entertainment'), ('categorynames', 'music industry')]
[('trademarkname', '24 Hour Fitness'), ('categorynames', 'fitness industry')]
[('trademarkname', '3M'), ('categorynames', 'conglomerate')]
[('trademarkname', '4 Asa U Kući'), ('categorynames', 'music industry')]
[('trademarkname', '430 West Records'), ('categorynames', 'music')]
[('trademarkname', '7-Eleven'), ('categorynames', 'convenience store')]
[('trademarkname', '88rising'), ('categorynames', 'music industry')]
[('trademarkname', '8D Creative'), ('categorynames', 'music industry')]
[('trademarkname', '960 Music Group'), ('categorynames', 'entertainm

40

## Categories or types

In [42]:
# categories or types
queryString = """
SELECT ?trademarkname ?countryname 
(GROUP_CONCAT(DISTINCT ?categoryname; SEPARATOR=", ") AS ?categorynames) 
WHERE { 
    
    ?trademark wdt:P31/wdt:P279* wd:Q167270  . 
    ?trademark wdt:P452 ?category .
    ?trademark wdt:P17 ?country .
    # Labels
    ?trademark <http://schema.org/name> ?trademarkname .
    ?category <http://schema.org/name> ?categoryname .
    ?country <http://schema.org/name> ?countryname .
    FILTER regex(?categoryname, "music", "i") 
} 
ORDER BY ?trademarkname
LIMIT 40
"""

print("Results")
run_query(queryString)

Results
[('trademarkname', '100PROmotion'), ('countryname', 'Czech Republic'), ('categorynames', 'music industry')]
[('trademarkname', '1333 Entertainment'), ('countryname', 'United Kingdom'), ('categorynames', 'music industry')]
[('trademarkname', '430 West Records'), ('countryname', 'United States of America'), ('categorynames', 'music')]
[('trademarkname', '88rising'), ('countryname', 'United States of America'), ('categorynames', 'music industry')]
[('trademarkname', '8D Creative'), ('countryname', 'South Korea'), ('categorynames', 'music industry')]
[('trademarkname', 'AWAL'), ('countryname', 'United Kingdom'), ('categorynames', 'music industry')]
[('trademarkname', 'Abridged Pause Recordings'), ('countryname', 'Canada'), ('categorynames', 'music industry')]
[('trademarkname', 'Abyss Company'), ('countryname', 'South Korea'), ('categorynames', 'music industry')]
[('trademarkname', 'Action Entertainment'), ('countryname', 'United States of America'), ('categorynames', 'music indust

40

## What are the types of trademarks, how many trademarks exist for each type?

#### Count the number of trademark that every category has.

In [43]:
# Count the number of trademark that every category has.
queryString = """
SELECT ?categoryname  (COUNT (DISTINCT ?trademark) AS ?counttrademark) 
WHERE { 
    
    ?trademark wdt:P31/wdt:P279* wd:Q167270  . 
    ?trademark wdt:P452 ?category .
    # Labels
   # ?trademark <http://schema.org/name> ?trademarkname .
    ?category <http://schema.org/name> ?categoryname .
   
} 
GROUP BY ?categoryname
ORDER BY DESC(?counttrademark)
LIMIT 40
"""

print("Results")
run_query(queryString)

Results
[('categoryname', 'retail'), ('counttrademark', '795')]
[('categoryname', 'automotive industry'), ('counttrademark', '504')]
[('categoryname', 'consumer electronics'), ('counttrademark', '227')]
[('categoryname', 'brewing'), ('counttrademark', '185')]
[('categoryname', 'music industry'), ('counttrademark', '184')]
[('categoryname', 'restaurant'), ('counttrademark', '175')]
[('categoryname', 'gastronomy'), ('counttrademark', '124')]
[('categoryname', 'clothing industry'), ('counttrademark', '98')]
[('categoryname', 'food industry'), ('counttrademark', '82')]
[('categoryname', 'entertainment'), ('counttrademark', '81')]
[('categoryname', 'fast food'), ('counttrademark', '73')]
[('categoryname', 'music'), ('counttrademark', '69')]
[('categoryname', 'video game industry'), ('counttrademark', '63')]
[('categoryname', 'hospitality industry'), ('counttrademark', '54')]
[('categoryname', 'food and tobacco industry'), ('counttrademark', '54')]
[('categoryname', 'phonographic industry'),

40

## What companies have the largest number of trademarks? 

In [44]:
# entertainment trademarks
queryString = """
SELECT ?groupname (COUNT (DISTINCT ?trademark) AS ?counttrademark) 
WHERE { 
    ?group ?relation ?trademark .
    ?trademark wdt:P31/wdt:P279* wd:Q167270  . 
    # Labels
    ?group <http://schema.org/name> ?groupname .
    FILTER regex(?groupname, "group", "i") 
} 
GROUP BY ?groupname
ORDER BY DESC(?counttrademark)
LIMIT 20
"""

print("Results")
run_query(queryString)

Results
[('groupname', 'Warner Music Group'), ('counttrademark', '52')]
[('groupname', 'Universal Music Group'), ('counttrademark', '50')]
[('groupname', 'Casino Group'), ('counttrademark', '10')]
[('groupname', 'Avex Group'), ('counttrademark', '9')]
[('groupname', 'InterContinental Hotels Group plc'), ('counttrademark', '9')]
[('groupname', 'Kraft Foods Group'), ('counttrademark', '9')]
[('groupname', 'Radisson Hotel Group'), ('counttrademark', '8')]
[('groupname', 'MTY Food Group'), ('counttrademark', '8')]
[('groupname', 'S Group'), ('counttrademark', '8')]
[('groupname', 'Minor Food Group'), ('counttrademark', '6')]
[('groupname', 'Campari Group'), ('counttrademark', '5')]
[('groupname', 'Kabad Music Group'), ('counttrademark', '5')]
[('groupname', 'Disney Music Group'), ('counttrademark', '5')]
[('groupname', 'Biogroup'), ('counttrademark', '5')]
[('groupname', 'Frasers Group'), ('counttrademark', '5')]
[('groupname', 'REWE Group'), ('counttrademark', '5')]
[('groupname', 'Carlsb

20

 ### Analyze the number of trademarks across types, companies, and countries

##### How many U.S. companies hold trademarks? In which sectors?

In [45]:
# categories or types
queryString = """
SELECT ?trademarkname
(GROUP_CONCAT(DISTINCT ?groupname; SEPARATOR=", ") AS ?groupnames)
WHERE { 
    ?group ?relation ?trademark .
    ?trademark wdt:P31/wdt:P279* wd:Q167270  . 
    ?trademark wdt:P452 ?category .
    ?trademark wdt:P17 ?country .
    # Labels
    ?trademark <http://schema.org/name> ?trademarkname .
    ?category <http://schema.org/name> ?categoryname .
    ?country <http://schema.org/name> ?countryname .
    ?group <http://schema.org/name> ?groupname .
    FILTER regex(?groupname, "group", "i") 
    FILTER regex(?countryname, "United States of America", "i") 
} 
LIMIT 40
"""

print("Results")
run_query(queryString)

Results
[('trademarkname', 'Burger King'), ('groupnames', 'Carrols Restaurant Group, Minor Food Group')]
[('trademarkname', 'Warner Music Group'), ('groupnames', 'Asgard (groupe), Atlantic Records Group, Category:Warner Music Group, Category:Warner Music Group albums, Category:Warner Music Group artists, Cosmos Music Group, Groupie, Maybach Music Group, Parlophone Label Group, Silent Majority Group, X5 Music Group')]
[('trademarkname', 'Alternative Distribution Alliance'), ('groupnames', 'Warner Music Group')]
[('trademarkname', 'National Amusements'), ('groupnames', 'Paramount Stations Group')]
[('trademarkname', 'Holiday Inn'), ('groupnames', 'InterContinental Hotels Group plc')]
[('trademarkname', 'Universal Music Group Nashville'), ('groupnames', 'Universal Music Group')]
[('trademarkname', 'Motel 6'), ('groupnames', 'The Blackstone Group')]
[('trademarkname', 'A&M Records'), ('groupnames', 'November Group')]
[('trademarkname', "Swensen's"), ('groupnames', 'Minor Food Group')]
[('t

39

In [46]:
## How many U.S. companies hold trademarks? 
queryString = """
SELECT (COUNT (DISTINCT ?group) AS ?countgroup) 
WHERE { 
    ?group ?relation ?trademark .
    ?trademark wdt:P31/wdt:P279* wd:Q167270  . 
    #?trademark wdt:P452 ?category .
    ?trademark wdt:P17 ?country .
    # Labels
    ?trademark <http://schema.org/name> ?trademarkname .
    #?category <http://schema.org/name> ?categoryname .
    ?country <http://schema.org/name> ?countryname .
    ?group <http://schema.org/name> ?groupname .
    FILTER regex(?groupname, "group", "i") 
    FILTER regex(?countryname, "United States of America", "i") 
} 

"""

print("Results")
run_query(queryString)

Results
[('countgroup', '137')]


1

In [47]:
## In which sectors are these companies?
queryString = """
SELECT ?categoryname (COUNT (DISTINCT ?group) AS ?countgroup) 
WHERE { 
    ?group ?relation ?trademark .
    ?trademark wdt:P31/wdt:P279* wd:Q167270  . 
    ?trademark wdt:P452 ?category .
    ?trademark wdt:P17 ?country .
    # Labels
    ?trademark <http://schema.org/name> ?trademarkname .
    ?category <http://schema.org/name> ?categoryname .
    ?country <http://schema.org/name> ?countryname .
    ?group <http://schema.org/name> ?groupname .
    FILTER regex(?groupname, "group", "i") 
    FILTER regex(?countryname, "United States of America", "i") 
} 
GROUP BY ?categoryname
ORDER BY DESC (?countgroup)
"""

print("Results")
run_query(queryString)

Results
[('categoryname', 'music industry'), ('countgroup', '34')]
[('categoryname', 'phonographic industry'), ('countgroup', '14')]
[('categoryname', 'show business'), ('countgroup', '7')]
[('categoryname', 'entertainment industry'), ('countgroup', '7')]
[('categoryname', 'mobile phone industry'), ('countgroup', '3')]
[('categoryname', 'digital distribution'), ('countgroup', '3')]
[('categoryname', 'software industry'), ('countgroup', '3')]
[('categoryname', 'electronics'), ('countgroup', '3')]
[('categoryname', 'gastronomy'), ('countgroup', '3')]
[('categoryname', 'information technology'), ('countgroup', '3')]
[('categoryname', 'consumer electronics'), ('countgroup', '3')]
[('categoryname', 'real estate investment trust'), ('countgroup', '2')]
[('categoryname', 'industry'), ('countgroup', '2')]
[('categoryname', 'film industry'), ('countgroup', '2')]
[('categoryname', 'hotel industry'), ('countgroup', '1')]
[('categoryname', 'early childhood education'), ('countgroup', '1')]
[('cate

33

### In which sector there is the highest number of trademarks?  

In [48]:
## In which sectors?
queryString = """
SELECT ?categoryname (COUNT (DISTINCT ?trademark) AS ?counttrademark) 
WHERE { 
    ?group ?relation ?trademark .
    ?trademark wdt:P31/wdt:P279* wd:Q167270  . 
    ?trademark wdt:P452 ?category .
    ?trademark wdt:P17 ?country .
    # Labels
    ?trademark <http://schema.org/name> ?trademarkname .
    ?category <http://schema.org/name> ?categoryname .
    ?country <http://schema.org/name> ?countryname .
    ?group <http://schema.org/name> ?groupname .
    FILTER regex(?groupname, "group", "i") 
    FILTER regex(?countryname, "United States of America", "i") 
} 
GROUP BY ?categoryname
ORDER BY DESC (?counttrademark)
LIMIT 1
"""

print("Results")
run_query(queryString)

Results
[('categoryname', 'music industry'), ('counttrademark', '12')]


1

##### How many people hold or are connected to trademarks? In which role?   

In [49]:
queryString = """
SELECT ?trademarkname ?counthuman WHERE {
{
SELECT ?trademarkname (COUNT (DISTINCT ?human) AS ?counthuman)
WHERE { 
    ?human ?relation ?trademark .
    ?trademark wdt:P31/wdt:P279* wd:Q167270  . 
    ?human wdt:P31 wd:Q5 .
        # Labels
    ?human <http://schema.org/name> ?humanname .
    ?trademark <http://schema.org/name> ?trademarkname .
    
} 

GROUP BY ?trademarkname 
} 

}
ORDER BY DESC (?counthuman) 


LIMIT 40
"""

print("Results")
run_query(queryString)

Results
[('trademarkname', 'Columbia Records'), ('counthuman', '1260')]
[('trademarkname', 'Universal Music Group'), ('counthuman', '1006')]
[('trademarkname', 'Atlantic Records'), ('counthuman', '736')]
[('trademarkname', 'Sony Music'), ('counthuman', '732')]
[('trademarkname', 'Capitol Records'), ('counthuman', '663')]
[('trademarkname', 'EMI'), ('counthuman', '644')]
[('trademarkname', 'RCA Records'), ('counthuman', '540')]
[('trademarkname', 'Epic Records'), ('counthuman', '516')]
[('trademarkname', 'Decca'), ('counthuman', '496')]
[('trademarkname', 'Warner Bros. Records'), ('counthuman', '488')]
[('trademarkname', 'Warner Music Group'), ('counthuman', '441')]
[('trademarkname', 'Island Records'), ('counthuman', '384')]
[('trademarkname', 'Red Bull'), ('counthuman', '368')]
[('trademarkname', 'Polydor Records'), ('counthuman', '344')]
[('trademarkname', 'Mercury Records'), ('counthuman', '309')]
[('trademarkname', 'King Records'), ('counthuman', '289')]
[('trademarkname', 'Blue No

40

In [50]:
queryString = """
SELECT ?humannames ?relationname ?trademarkname
WHERE {
    ?human ?relation ?trademark .
    ?trademark wdt:P31/wdt:P279* wd:Q167270  . 
    ?human wdt:P31 wd:Q5 .
        # Labels
    ?human <http://schema.org/name> ?humanname .
    ?trademark <http://schema.org/name> ?trademarkname .
    ?relation <http://schema.org/name> ?relationname .
}
GROUP BY ?trademarkname 
ORDER BY DESC (?trademarkname) 
LIMIT 40
"""

print("Results")
run_query(queryString)

Results
[('relationname', 'record label'), ('trademarkname', 'İstanbul Plak')]
[('relationname', 'record label'), ('trademarkname', 'İdobay Müzik')]
[('relationname', 'record label'), ('trademarkname', 'él')]
[('relationname', 'record label'), ('trademarkname', 'Übersee Records')]
[('relationname', 'record label'), ('trademarkname', 'Øra fonogram')]
[('relationname', 'record label'), ('trademarkname', 'Élan Recordings')]
[('relationname', 'record label'), ('trademarkname', 'will.i.am Music Group')]
[('relationname', 'record label'), ('trademarkname', 'wavemob')]
[('relationname', 'employer'), ('trademarkname', 'uBreakiFix')]
[('relationname', 'chief executive officer'), ('trademarkname', 'uBreakiFix')]
[('relationname', 'record label'), ('trademarkname', 'tearbridge production')]
[('relationname', 'record label'), ('trademarkname', 'taâlem')]
[('relationname', 'record label'), ('trademarkname', 'swim ~')]
[('relationname', 'record label'), ('trademarkname', 'self-released')]
[('relatio

40

In [51]:
queryString = """
SELECT ?relationname ?trademarkname
(GROUP_CONCAT(DISTINCT ?humanname; SEPARATOR=", ") AS ?humannames)
WHERE {
    ?human ?relation ?trademark .
    ?trademark wdt:P31/wdt:P279* wd:Q167270  . 
    ?human wdt:P31 wd:Q5 .
        # Labels
    ?human <http://schema.org/name> ?humanname .
    ?trademark <http://schema.org/name> ?trademarkname .
    ?relation <http://schema.org/name> ?relationname .
}
ORDER BY DESC (?trademarkname) 
LIMIT 40
"""

print("Results")
run_query(queryString)

Results
[('relationname', 'record label'), ('trademarkname', 'İstanbul Plak'), ('humannames', 'Ajda Pekkan, Ebru Gündeş, Özlem Tekin')]
[('relationname', 'record label'), ('trademarkname', 'İdobay Müzik'), ('humannames', 'Ahmet Koç, Ceylan, Ebru Yaşar, Günel, Harun Kolçak, Müslüm Gürses, Nuri Sesigüzel, Onur Akın, Petek Dinçöz, Soner Arıca, Yıldız Tilbe, Ömer Danış, İsmail Türüt')]
[('relationname', 'record label'), ('trademarkname', 'él'), ('humannames', 'Louis Philippe, Momus')]
[('relationname', 'record label'), ('trademarkname', 'Übersee Records'), ('humannames', 'Dario Espinosa, Dr. Shenka, Felipe Bustamante, Gorri, Hiram Paniagua, Leonel Rosales, Missael, Tanis')]
[('relationname', 'record label'), ('trademarkname', 'Øra fonogram'), ('humannames', 'Hanna Paulsberg')]
[('relationname', 'record label'), ('trademarkname', 'Élan Recordings'), ('humannames', 'Santiago Rodriguez')]
[('relationname', 'record label'), ('trademarkname', 'will.i.am Music Group'), ('humannames', 'Jermain Ja

40

In [52]:
## find apple products 
queryString = """
SELECT DISTINCT ?o ?oname
WHERE { 

    wd:Q312 wdt:P1056 ?o .
    
    # Labels

    ?o <http://schema.org/name> ?oname .
    #FILTER regex(?pname, "company", "i") 
} 
LIMIT 50
"""

print("Results")
run_query(queryString)


Results
[('o', 'http://www.wikidata.org/entity/Q26833946'), ('oname', 'iPhone\xa07 Plus')]
[('o', 'http://www.wikidata.org/entity/Q39510168'), ('oname', 'iPhone X')]
[('o', 'http://www.wikidata.org/entity/Q20962515'), ('oname', 'Apple Pencil')]
[('o', 'http://www.wikidata.org/entity/Q18010891'), ('oname', 'iPhone 6 Plus')]
[('o', 'http://www.wikidata.org/entity/Q20966885'), ('oname', 'iPhone 6S Plus')]
[('o', 'http://www.wikidata.org/entity/Q214276'), ('oname', 'MacBook Pro')]
[('o', 'http://www.wikidata.org/entity/Q308869'), ('oname', 'Mac Pro')]
[('o', 'http://www.wikidata.org/entity/Q486053'), ('oname', 'MacBook')]
[('o', 'http://www.wikidata.org/entity/Q270285'), ('oname', 'Apple TV')]
[('o', 'http://www.wikidata.org/entity/Q300859'), ('oname', 'iPhone 3GS')]
[('o', 'http://www.wikidata.org/entity/Q300890'), ('oname', 'iPhone 4')]
[('o', 'http://www.wikidata.org/entity/Q300947'), ('oname', 'iPhone 3G')]
[('o', 'http://www.wikidata.org/entity/Q308871'), ('oname', 'Mac Mini')]
[('o',

44

In [53]:
## count apple products 
queryString = """
SELECT (COUNT (DISTINCT ?product) AS ?countproduct) 
WHERE { 

    wd:Q312 wdt:P1056 ?product .
    
    # Labels

    ?product <http://schema.org/name> ?productname .
    #FILTER regex(?pname, "company", "i") 
} 
"""

print("Results")
run_query(queryString)


Results
[('countproduct', '44')]


1

In [54]:

queryString = """
SELECT ?p ?pname ?o ?oname
WHERE { 

    wd:Q26833946 ?p ?o .
    
    # Labels
    ?p <http://schema.org/name> ?pname .
    ?o <http://schema.org/name> ?oname .
} 
LIMIT 50
"""

print("Results")
run_query(queryString)


Results
[('p', 'http://www.wikidata.org/prop/direct/P306'), ('pname', 'operating system'), ('o', 'http://www.wikidata.org/entity/Q61497975'), ('oname', 'iOS 13')]
[('p', 'http://www.wikidata.org/prop/direct/P880'), ('pname', 'CPU'), ('o', 'http://www.wikidata.org/entity/Q25385678'), ('oname', 'Apple A10 Fusion')]
[('p', 'http://www.wikidata.org/prop/direct/P156'), ('pname', 'followed by'), ('o', 'http://www.wikidata.org/entity/Q39510168'), ('oname', 'iPhone X')]
[('p', 'http://www.wikidata.org/prop/direct/P2033'), ('pname', 'category for pictures taken with camera'), ('o', 'http://www.wikidata.org/entity/Q63630069'), ('oname', 'Category:Taken with iPhone 7 Plus')]
[('p', 'http://www.wikidata.org/prop/direct/P5009'), ('pname', 'complies with'), ('o', 'http://www.wikidata.org/entity/Q67226021'), ('oname', 'IEC 60529 Degrees of protection provided by enclosures (IP Code)')]
[('p', 'http://www.wikidata.org/prop/direct/P2935'), ('pname', 'connector'), ('o', 'http://www.wikidata.org/entity/Q

17

In [55]:
queryString = """
SELECT ?p ?pname ?o ?oname
WHERE { 

     ?p ?o wd:Q26833946 .
    
    # Labels
    ?p <http://schema.org/name> ?pname .
    ?o <http://schema.org/name> ?oname .
} 
LIMIT 50
"""

print("Results")
run_query(queryString)


Results
[('p', 'http://www.wikidata.org/entity/Q312'), ('pname', 'Apple Inc.'), ('o', 'http://www.wikidata.org/prop/direct/P1056'), ('oname', 'product or material produced')]
[('p', 'http://www.wikidata.org/entity/Q574890'), ('pname', 'Retina Display'), ('o', 'http://www.wikidata.org/prop/direct/P1535'), ('oname', 'used by')]
[('p', 'http://www.wikidata.org/entity/Q39598190'), ('pname', 'iPhone\xa08 Plus'), ('o', 'http://www.wikidata.org/prop/direct/P155'), ('oname', 'follows')]
[('p', 'http://www.wikidata.org/entity/Q20966885'), ('pname', 'iPhone 6S Plus'), ('o', 'http://www.wikidata.org/prop/direct/P156'), ('oname', 'followed by')]
[('p', 'http://www.wikidata.org/entity/Q67064073'), ('pname', 'iPhone 7 Plus, model A1786'), ('o', 'http://www.wikidata.org/prop/direct/P279'), ('oname', 'subclass of')]
[('p', 'http://www.wikidata.org/entity/Q66816666'), ('pname', 'iPhone 7 Plus, model A1784'), ('o', 'http://www.wikidata.org/prop/direct/P279'), ('oname', 'subclass of')]
[('p', 'http://www

12

In [56]:
queryString = """
SELECT ?cpu ?cpuname
WHERE { 

    wd:Q26833946 wdt:P880 ?cpu .
    
    # Labels
    ?cpu <http://schema.org/name> ?cpuname .
} 
LIMIT 50
"""

print("Results")
run_query(queryString)


Results
[('cpu', 'http://www.wikidata.org/entity/Q25385678'), ('cpuname', 'Apple A10 Fusion')]


1

In [57]:
## find apple products + their cpu info. 
queryString = """
SELECT ?cpuname (GROUP_CONCAT(DISTINCT ?productname; SEPARATOR=", ") AS ?productnames)
WHERE { 

    wd:Q312 wdt:P1056 ?product .
    ?product wdt:P880 ?cpu .
    
    # Labels
    ?cpu <http://schema.org/name> ?cpuname .
    ?product <http://schema.org/name> ?productname .
} 
LIMIT 50
"""

print("Results")
run_query(queryString)


Results
[('cpuname', 'Apple A10 Fusion'), ('productnames', 'iPad, iPhone 7, iPhone\xa07 Plus, iPod Touch')]
[('cpuname', 'Apple A9'), ('productnames', 'iPad, iPhone 6S, iPhone 6S Plus')]
[('cpuname', 'Apple A8'), ('productnames', 'HomePod, iPad, iPhone 6, iPhone 6 Plus, iPod Touch')]
[('cpuname', 'Apple A5X'), ('productnames', 'iPad')]
[('cpuname', 'Apple Motion Coprocessor'), ('productnames', 'iPhone 5S')]
[('cpuname', 'Apple A10X Fusion'), ('productnames', 'iPad')]
[('cpuname', 'Apple A8X'), ('productnames', 'iPad')]
[('cpuname', 'Apple A12 Bionic'), ('productnames', 'iPad, iPhone 11 Pro Max, iPhone XR, iPhone XS')]
[('cpuname', 'Apple A6'), ('productnames', 'iPhone 5, iPhone 5C')]
[('cpuname', 'ARM Cortex-A8'), ('productnames', 'iPhone 3GS')]
[('cpuname', 'Apple A7'), ('productnames', 'iPad, iPhone 5S')]
[('cpuname', 'Apple A4'), ('productnames', 'iPad, iPhone 4, iPod Touch')]
[('cpuname', 'Apple A5'), ('productnames', 'iPad, iPhone 4S, iPod Touch')]
[('cpuname', 'Apple A11 Bionic')

16